In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 200

# Reading in CSVs

In [3]:
#Reading in CSVs from downloaded kaggle dataset
calendar_summary = pd.read_csv('data/calendar_summary.csv')
listings = pd.read_csv('data/listings.csv')
listings_summary = pd.read_csv('data/listings_summary.csv')
neighbourhoods = pd.read_csv('data/neighbourhoods.csv')
reviews = pd.read_csv('data/reviews.csv')
reviews_summary = pd.read_csv('data/reviews_summary.csv')

# Data Exploration

In [4]:
"""
There's no schema associated with this dataset, so we have 
to determine how these csvs are connected
"""

"\nThere's no schema associated with this dataset, so we have \nto determine how these csvs are connected\n"

## calendar_summary exploration

In [5]:
calendar_summary.head()

,listing_id,date,available,price
0,2015,2019-09-10,f,NaN
1,2015,2019-09-09,f,NaN
2,2015,2019-09-08,f,NaN
3,2015,2019-09-07,f,NaN
4,2015,2019-09-06,f,NaN


In [6]:
calendar_summary['listing_id'].value_counts()

29603721    365
13585558    365
15801211    365
16235439    365
27244206    365
           ... 
7890338     365
17263170    365
29516515    365
25693102    365
21838247    365
Name: listing_id, Length: 22552, dtype: int64

In [7]:
calendar_summary.describe(exclude='number')

,date,available,price
count,8231480,8231480,1800841
unique,367,2,684
top,2018-11-25,f,$50.00
freq,22552,6430639,68088


In [8]:
calendar_summary['available'].value_counts()

f    6430639
t    1800841
Name: available, dtype: int64

In [9]:
calendar_summary.isnull().sum()

listing_id          0
date                0
available           0
price         6430639
dtype: int64

In [11]:
#calendar_summary.sort_values(by='date')

In [12]:

"""
Looking at the columns:

'listings_id' looks like there are 22552 unique listings.

'date' is odd that there are 367 unique values rather than 365
I'll have to look into that.
Looked into it. It goes from 2018-11-07 to 2019-11-08, which is 367 days.
It's not a perfect year, which is just fiiiiine.
For any prediction, I won't take the year into account, just the month/day.

'available' has two values, t and f. Probably means True and False.
mostly false. So the majority of the time the units are unavailable.

price is obiously price.
Note that there are only prices for dates that the unit is available
Null values inhabit all of the rows where the unit is occpied, which makes 
sense.
We can determine which price the unit was rented at by looking at the 
earliest price before occupation. Not sure how useful it'll be.
"""

"\nLooking at the columns:\n\n'listings_id' looks like there are 22552 unique listings.\n\n'date' is odd that there are 367 unique values rather than 365\nI'll have to look into that.\nLooked into it. It goes from 2018-11-07 to 2019-11-08, which is 367 days.\nIt's not a perfect year, which is just fiiiiine.\nFor any prediction, I won't take the year into account, just the month/day.\n\n'available' has two values, t and f. Probably means True and False.\nmostly false. So the majority of the time the units are unavailable.\n\nprice is obiously price.\nNote that there are only prices for dates that the unit is available\nNull values inhabit all of the rows where the unit is occpied, which makes \nsense.\nWe can determine which price the unit was rented at by looking at the \nearliest price before occupation. Not sure how useful it'll be.\n"

calendar_summary columns

listing_id = The id for the listing. The same id for multiple rows

data = the date for that listing. Each listing has multiple dates

available = t/f (True/False) if the unit is available

price = the price, only if the unit is available. 
    We can determine which price the unit was rented at by looking at the 
    earliest price before occupation. Not sure how useful it'll be.

## listings exploration

In [13]:
listings.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2015,Berlin-Mitte Value! Quiet courtyard/very central,2217,Ian,Mitte,Brunnenstr. Süd,52.534537,13.402557,Entire home/apt,60,4,118,2018-10-28,3.76,4,141
1,2695,Prenzlauer Berg close to Mauerpark,2986,Michael,Pankow,Prenzlauer Berg Nordwest,52.548513,13.404553,Private room,17,2,6,2018-10-01,1.42,1,0
2,3176,Fabulous Flat in great Location,3718,Britta,Pankow,Prenzlauer Berg Südwest,52.534996,13.417579,Entire home/apt,90,62,143,2017-03-20,1.25,1,220
3,3309,BerlinSpot Schöneberg near KaDeWe,4108,Jana,Tempelhof - Schöneberg,Schöneberg-Nord,52.498855,13.349065,Private room,26,5,25,2018-08-16,0.39,1,297
4,7071,BrightRoom with sunny greenview!,17391,Bright,Pankow,Helmholtzplatz,52.543157,13.415091,Private room,42,2,197,2018-11-04,1.75,1,26


In [14]:
listings['host_id'].value_counts().shape

(19180,)

In [15]:
listings['neighbourhood_group'].value_counts().shape

(12,)

In [16]:
listings['room_type'].value_counts().shape

(3,)

In [17]:
"""
Looking at the columns:

'id' = This is the same as 'listing_id' in calendar_summary
'host_id' = the unique id for each host. There are 19180 values here,
    which means a lot of hosts have multiple units Not useful
'host_name' = Looks like the first name for each host. Not useful
'neighbourhood_group' = the 12 groupings of Berlin neighbourhoods
'neighbourhood' = Which neighbourhood the unit is located in
'latitude' = 
'longitude' = what you'd expect
'room_type' = which of 3 room types is the unit
'price' = integer of price
'mimimum_nights' = integer of minimum nights. Not useful.
'number_of_reviews' = that. Not useful.
'reviews_per_month' = also that. Not useful
'calculated_host_listings_count' = Number of listings the host of that unit has.
    Not useful.
'availability_365' = How many days of the year that unit is available. Not useful.

    
"""

"\nLooking at the columns:\n\n'id' = This is the same as 'listing_id' in calendar_summary\n'host_id' = the unique id for each host. There are 19180 values here,\n    which means a lot of hosts have multiple units Not useful\n'host_name' = Looks like the first name for each host. Not useful\n'neighbourhood_group' = the 12 groupings of Berlin neighbourhoods\n'neighbourhood' = Which neighbourhood the unit is located in\n'latitude' = \n'longitude' = what you'd expect\n'room_type' = which of 3 room types is the unit\n'price' = integer of price\n'mimimum_nights' = integer of minimum nights. Not useful.\n'number_of_reviews' = that. Not useful.\n'reviews_per_month' = also that. Not useful\n'calculated_host_listings_count' = Number of listings the host of that unit has.\n    Not useful.\n'availability_365' = How many days of the year that unit is available. Not useful.\n\n    \n"

## listings_summary exploration

In [18]:
listings_summary.head(1)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,2015,https://www.airbnb.com/rooms/2015,20181107122246,2018-11-07,Berlin-Mitte Value! Quiet courtyard/very central,Great location! 30 of 75 sq meters. This wood...,A+++ location! This „Einliegerwohnung“ is an e...,Great location! 30 of 75 sq meters. This wood...,none,It is located in the former East Berlin area o...,"This is my home, not a hotel. I rent out occas...","Close to U-Bahn U8 and U2 (metro), Trams M12, ...","Simple kitchen/cooking, refrigerator, microwav...",Always available,"No parties No events No pets No smoking, not e...",NaN,NaN,https://a0.muscache.com/im/pictures/260fd609-7...,NaN,2217,https://www.airbnb.com/users/show/2217,Ian,2008-08-18,"Key Biscayne, Florida, United States",Believe in sharing economy.,within an hour,96%,NaN,t,https://a0.muscache.com/im/pictures/21428a22-4...,https://a0.muscache.com/im/pictures/21428a22-4...,Mitte,4.0,4.0,"['email', 'phone', 'reviews', 'jumio', 'offlin...",t,t,"Berlin, Berlin, Germany",Mitte,Brunnenstr. Süd,Mitte,Berlin,Berlin,10119,Berlin,"Berlin, Germany",DE,Germany,52.534537,13.402557,f,Guesthouse,Entire home/apt,3,1.0,1.0,2.0,Real Bed,"{TV,""Cable TV"",Wifi,Kitchen,Gym,Heating,""Famil...",NaN,$60.00,NaN,NaN,$200.00,$30.00,1,$28.00,4,1125,3 months ago,t,0,21,51,141,2018-11-07,118,2016-04-11,2018-10-28,93.0,10.0,9.0,10.0,10.0,10.0,9.0,t,NaN,NaN,f,f,strict_14_with_grace_period,f,f,4,3.76


In [ ]:
"""
This seems to contain the most useful information. 
Lets go with this one
"""

## neighbourhoods exploration

In [12]:
neighbourhoods.head()

,neighbourhood_group,neighbourhood
0,Charlottenburg-Wilm.,Barstraße
1,Charlottenburg-Wilm.,Charlottenburg Nord
2,Charlottenburg-Wilm.,Düsseldorfer Straße
3,Charlottenburg-Wilm.,Forst Grunewald
4,Charlottenburg-Wilm.,Grunewald


## reviews exploration

In [49]:
reviews.head()

,listing_id,date
0,2015,2016-04-11
1,2015,2016-04-15
2,2015,2016-04-26
3,2015,2016-05-10
4,2015,2016-05-14


## reviews_summary exploration

In [14]:
reviews_summary.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2015,69544350,2016-04-11,7178145,Rahel,Mein Freund und ich hatten gute gemütliche vie...
1,2015,69990732,2016-04-15,41944715,Hannah,Jan was very friendly and welcoming host! The ...
2,2015,71605267,2016-04-26,30048708,Victor,Un appartement tres bien situé dans un quartie...
3,2015,73819566,2016-05-10,63697857,Judy,"It is really nice area, food, park, transport ..."
4,2015,74293504,2016-05-14,10414887,Romina,"Buena ubicación, el departamento no está orden..."
